# HWDP

$ x = \frac{\sqrt{z}}{y} $

In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
db= pd.read_csv(r'filtered_data.csv')

In [4]:
db.head()

,Unnamed: 0,Event,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,Game
0,0,Rated Bullet game,0-1,1079,1071,-30,10,B00,Owen Defense,60+0,Time forfeit,1. e4 { [%eval 0.12] [%clk 0:01:00] } 1... b6?...
1,1,Rated Blitz game,0-1,1888,1796,-13,13,C62,Ruy Lopez: Steinitz Defense,300+0,Normal,1. e4 { [%eval 0.12] [%clk 0:05:00] } 1... e5 ...
2,2,Rated Rapid game,0-1,1090,1156,-9,11,B20,Sicilian Defense,600+0,Normal,1. e4 { [%eval 0.12] [%clk 0:10:00] } 1... c5 ...
3,3,Rated Bullet game,0-1,1091,1184,-8,9,C20,"King's Pawn Game: Wayward Queen Attack, Kiddie...",60+0,Time forfeit,1. e4 { [%eval 0.12] [%clk 0:01:00] } 1... e5 ...
4,4,Rated Classical game,1-0,1672,1699,11,-11,C64,"Ruy Lopez: Classical Variation, Central Variation",900+15,Normal,1. e4 { [%eval 0.12] [%clk 0:15:00] } 1... e5 ...


In [5]:
db.describe()

,Unnamed: 0,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff
count,31461.000000,31461.000000,31461.000000,31461.000000,31461.000000
mean,15730.000000,1503.446903,1503.921967,4.022536,2.654684
std,9082.152746,321.223393,320.695908,30.933538,29.354305
min,0.000000,800.000000,800.000000,-437.000000,-615.000000
25%,7865.000000,1266.000000,1267.000000,-10.000000,-10.000000
50%,15730.000000,1500.000000,1500.000000,3.000000,-2.000000
75%,23595.000000,1725.000000,1724.000000,11.000000,11.000000
max,31460.000000,2748.000000,2667.000000,460.000000,598.000000


In [4]:
# domyślnie bez [:5]
a = db.Game
a

0        1. e4 { [%eval 0.12] [%clk 0:01:00] } 1... b6?...
1        1. e4 { [%eval 0.12] [%clk 0:05:00] } 1... e5 ...
2        1. e4 { [%eval 0.12] [%clk 0:10:00] } 1... c5 ...
3        1. e4 { [%eval 0.12] [%clk 0:01:00] } 1... e5 ...
4        1. e4 { [%eval 0.12] [%clk 0:15:00] } 1... e5 ...
                               ...                        
31456    1. d4 { [%eval 0.25] [%clk 0:05:00] } 1... d5 ...
31457    1. e4 { [%eval 0.12] [%clk 0:10:00] } 1... e5 ...
31458    1. e4 { [%eval 0.12] [%clk 0:10:00] } 1... e5 ...
31459    1. e4 { [%eval 0.12] [%clk 0:10:00] } 1... e5 ...
31460    1. c4 { [%eval 0.1] [%clk 0:02:00] } 1... e5 {...
Name: Game, Length: 31461, dtype: object

In [16]:
def time_to_sec(hhmmss):
    return sum(x * int(t) for x, t in zip([3600, 60, 1], hhmmss.split(":")))

def derivative(value_list, step=1, reversed=False):
    v_copy = [it for it in value_list]
    for i in range(len(v_copy)-1, -1, -1):
        if i in np.arange(step):
            v_copy[i] = 0
        if i >= step:
            try:
                v_copy[i] = round(v_copy[i] - v_copy[i-step], 2)
            except TypeError:
                pass
    if reversed:
        v_copy = [-1 * it for it in v_copy]
    return v_copy

In [17]:
# na razie na a[0] -> zrobić fora dla wszystkich
# ev = re.findall(r"\%eval (.*?)\]",a[0])
# ev = [float(it) for it in ev]
#
# t_left = re.findall(r"\%clk (.*?)\]",a[0])
# t_left = [time_to_sec(it) for it in t_left]
#
# d_t = derivative(t_left, step=2, reversed=True)
# d_ev = [round(it,2) for it in derivative(ev, step=1, reversed=False)]
# ev
all_ev = []
all_d_ev = []
all_t_left = []
all_d_t = []
for j in range(len(a)):
    if j % 1000 == 0:
        print(j)
    ev = re.findall(r"\%eval (.*?)\]",a[j])
    for i in range(len(ev)):
        try:
            ev[i] = round(float(ev[i]),2)
        except ValueError:
            pass
    all_ev.append(ev)
    t_left = re.findall(r"\%clk (.*?)\]",a[j])
    t_left = [time_to_sec(it) for it in t_left]
    all_t_left.append(t_left)
    all_d_t.append(derivative(t_left, step=2, reversed=True))
    all_d_ev.append(derivative(ev, step=1, reversed=False))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


[3, 2, 1, 0]

In [18]:
db['Eval'] = all_ev
db['Eval_change'] = all_d_ev
db['Time'] = all_t_left
db['Time_change'] = all_d_t

In [19]:
db.head(1)


,Unnamed: 0,Event,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,Game,Eval,Eval_change,Time,Time_change
0,0,Rated Bullet game,0-1,1079,1071,-30,10,B00,Owen Defense,60+0,Time forfeit,1. e4 { [%eval 0.12] [%clk 0:01:00] } 1... b6?...,"[0.12, 0.62, 0.64, 0.62, 0.47, 0.54, 0.27, 0.6...","[0, 0.5, 0.02, -0.02, -0.15, 0.07, -0.27, 0.41...","[60, 60, 57, 60, 57, 60, 55, 57, 51, 55, 48, 5...","[0, 0, 3, 0, 0, 0, 2, 3, 4, 2, 3, 1, 2, 1, 1, ..."


In [21]:
db.iloc[10,-3]

[0,
 -0.1,
 0.11,
 0.13,
 -0.19,
 0.58,
 -0.26,
 0.19,
 -0.19,
 -0.04,
 0.29,
 -0.09,
 -0.41,
 0.34,
 -0.15,
 0.07,
 -0.09,
 1.92,
 -2.11,
 3.45,
 -3.32,
 1.28,
 0.02,
 -0.03,
 0.14,
 -0.26,
 0.27,
 -0.06,
 -0.03,
 0.77,
 -0.58,
 0.9,
 -0.39,
 1.74,
 -1.13,
 0.23,
 -1.01,
 0.02,
 -0.16,
 0.53,
 -0.02,
 -0.04,
 -0.21,
 0.16,
 -0.59,
 0.55,
 -0.06,
 1.52,
 -3.03,
 1.41,
 -0.15,
 9.61,
 -0.7,
 '#2',
 '#1',
 '#1']

In [24]:
db.to_csv(r'C:\Users\piotr\OneDrive\Documents\GitHub\Chess\Python\filtered_data_wt_eval.csv', mode='w')